Most of your work is related to how well this one single function performs:

```python
def predict(data):
  ...
```

To the untrained eye, this looks deceptively simple. Isn't this just one function? I can guarantee you you're about to develop a love-hate relationship with it!

In reality, when dealing with Deep Learning the `predict()` function can have a simple implementation:

```python
def predict(data):
  return data * weight
```

But what does `weight` contain? This is the "brains" of your model. Without good weight - you don't get to drink Corona on your favorite beach, washboard abs, and hotties around you.

### How can you get some good weights for yourself?

There are some ways to find good weight values, but let's start with something really dumb (a good baseline if you will) and improve on that (hopefully, in a non-linear fashion).

Let's say you need to predict the Tesla stock price. The current price is 420 USD. What will be the price in a minute?

In [ ]:
def predict(data, weight):
  return data * weight

Actually, this is your first model. Congratulations! Let's use it:

In [ ]:
current_stock_price = 420

prediction = predict(current_stock_price, weight=1.01)

f"Predicted Tesla stock price is {prediction} USD"

'Predicted Tesla stock price is 424.2 USD'

Yes, it works! Unfortunately, the real price is 425 USD. How can you use this information to make your model better?

One popular way is to start by measuring the error of the prediction:

In [ ]:
import torch
import torch.nn.functional as F

real_next_price = 425

error = F.mse_loss(
  input=torch.tensor(prediction), 
  target=torch.tensor(real_next_price)
)

error

tensor(0.6400)

Great, now you have a concrete goal - reduce the error to 0. We can continue with our dumb strategy of guessing weight values:

In [ ]:
weight = 1.01

change = 0.0001

target = torch.tensor(real_next_price)

for _ in range(1000):
  guess_up = torch.tensor(predict(current_stock_price, weight + change))
  guess_down = torch.tensor(predict(current_stock_price, weight - change))

  error_up = F.mse_loss(guess_up, target)
  error_down = F.mse_loss(guess_down, target)

  if error_up < error_down:
    weight += change
  else:
    weight -= change

We start with the current weight and try to reduce or increase it by a bit. We take whichever change has a smaller error and repeat.

What is the predicted stock price with the new model?

In [ ]:
predict(current_stock_price, weight)

425.0399999999999

This one looks much better. What about the error?

In [ ]:
F.mse_loss(
  input=torch.tensor(predict(current_stock_price, weight)),
  target=target
)

tensor(0.0016)

A drastic improvement compared to the first model. But can you really use this guessing method in practice? No!